We are building a Semantic search engine to find relavant job roles uing the job describtion or prompt

In [ ]:
ENDPOINT="https://localhost:9200/"
USERNAME="XXX"
PASSWORD="XXX"

In [ ]:
!pip install -U sentence-transformers


In [ ]:
! pip install transformers


In [ ]:
# Import libraries 

import pandas as pd
import numpy as np
import json
import os
import uuid

In [ ]:
pip install elasticsearch

In [ ]:

from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

import elasticsearch
from elasticsearch import Elasticsearch
from elasticsearch import helpers

from tqdm.auto import tqdm
tqdm.pandas()

In [ ]:
filepath = "data job posts.csv"

df  = pd.read_csv(filepath, engine='python', encoding='utf-8', error_bad_lines=False)



In [ ]:
df.shape

This is Sample class that will help to convert sentences into Token

In [ ]:
class Tokenizer(object):
    def __init__(self):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')

    def get_token(self, documents):
        sentences  = [documents]
        sentence_embeddings = self.model.encode(sentences)
        encod_np_array = np.array(sentence_embeddings)
        encod_list = encod_np_array.tolist()
        return encod_list[0]

Step 3: Creating Tokens
Lets create corups of vectors and use teh power of elastic search to Search vectors with semantics Search

In [ ]:
token_instance = Tokenizer()


In [ ]:
df = df.head(5000)
df = df.dropna(how='all')
df['vector'] = df['jobpost'].progress_apply(token_instance.get_token)

In [ ]:
elk_data = df.to_dict("records")

In [ ]:
pip install elasticsearch==7.17

In [ ]:
es = Elasticsearch(hosts=[ENDPOINT],  http_auth=(USERNAME, PASSWORD))
es.ping()

In [ ]:
for x in elk_data:
    try:
        _={
           "title":x.get("Title", ""),
            "company":x.get("Company", ""),
            "location":x.get("Location", ""),
            "salary":x.get("Salary", ""),
            "vector":x.get("vector", ""),
            "job_description":x.get("JobDescription", ""),

        }
        es.index(index='posting1',body=_)
    except Exception as e:pass

# Test For Semantics Search


In [ ]:
INPUT = input("Enter the Input Query ")
token_vector = token_instance.get_token(INPUT)

query  ={
  "_source": ["title", "job_description"], 
   "size":50,
   "query":{
      "bool":{
         "must":[
            {
               "knn":{
                  "vector":{
                     "vector":token_vector,
                     "k":20
                  }
               }
            }
         ]
      }
   }
}


res = es.search(index='posting1',
                size=50,
                body=query,
                request_timeout=55)

title = [x['_source']  for x in res['hits']['hits']] 
title